# PyCity Schools Analysis

Observed Trend 1: Higher Spending per student does not lead necessarily lead to higher scores. In fact, we see that the schools in the lower spending ranges, have higher average math and reading scores as well as higher pass rates.

Observed Trend 2: Smaller schools in size have higher average math and reading scores and higher math and reading pass rates while larger schools have lower scores and lower pass rates.

Observed Trend 3: Charter schools have higher average math and reading scores and have higher math and reading pass rates than District schools.

In [446]:
import pandas as pd
import numpy as np

In [447]:
schools_df = pd.read_csv("schools_complete.csv")
students_df = pd.read_csv("students_complete.csv")

# District Summary

In [448]:
# Count number of schools
total_schools = schools_df["name"].count()

# Count number of students
total_students = students_df["name"].count()

# Sum for total budget
total_budget = schools_df["budget"].sum()

# Average for Math score
avg_math_score = students_df["math_score"].mean()

# Average for Reading score
avg_reading_score = students_df["reading_score"].mean()

# Find location where Math score is "Pass", assuming passing is score of 65 and up
pass_math_df = students_df.loc[students_df["math_score"]>=65,["school","name","Student ID"]]
pass_math_df["pass_math"] = "Pass"
count_pass_math = pass_math_df["name"].count()
percent_pass_math = count_pass_math / total_students

# Find location where Reading score is "Pass", assuming passing is score of 65 and up
pass_reading_df = students_df.loc[students_df["reading_score"]>=65,["school","name","Student ID"]]
pass_reading_df["pass_reading"] = "Pass"
count_pass_reading = pass_reading_df["name"].count()
percent_pass_reading = count_pass_reading / total_students

# Overall Pass Rate is average of Math Pass Rate and Reading Pass Rate
avg_pass_rate = (percent_pass_math + percent_pass_reading) / 2

# Create dataframe for district summary table
district_summary = pd.DataFrame({"Total Schools":[total_schools],
                                       "Total Students":[total_students],
                                       "Total Budget":[total_budget],
                                       "Average Math Score":[avg_math_score],
                                       "Average Reading Score":[avg_reading_score],
                                       "% Passing Math":[percent_pass_math],
                                       "% Passing Reading":[percent_pass_reading],
                                       "Overall Passing Rate":[avg_pass_rate]})

# Format values
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:,.2f}".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:,.2f}".format)
district_summary["Overall Passing Rate"] = (district_summary["Overall Passing Rate"]*100).map("{:,.2f}%".format)

# Arrange column names in order of table display
col_arrange = ["Total Schools","Total Students","Total Budget","Average Math Score","Average Reading Score","Overall Passing Rate"]
district_summary = district_summary[col_arrange]

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,90.46%


# School Summary

In [449]:
# Merge students_df with pass_math_df and pass_reading_df
students2_df = pd.merge(pd.merge(students_df, pass_math_df, how="left", on=["school","name","Student ID"]),
                        pass_reading_df, how="left", on=["school","name","Student ID"])

# Merge new students datafram with schools_df
merge_table = pd.merge(students2_df,schools_df, how="left", left_on="school", right_on="name")

# Rename column to student_name and delete name_y
merge_table = merge_table.rename(columns={"name_x":"student_name"})
del merge_table["name_y"]

In [450]:
# Group by School Name and School Type
groupby_table = merge_table.groupby(["school","type"],as_index=False)

# Count Student Names and number Passing Math and Reading
schools_summ_count = groupby_table[["student_name","pass_math","pass_reading"]].count()

# Calculate for % Passing Math, Reading, and Overall Passing Rate
schools_summ_count["% Passing Math"] = ((schools_summ_count["pass_math"])/(schools_summ_count["student_name"]))*100
schools_summ_count["% Passing Reading"] = ((schools_summ_count["pass_reading"])/(schools_summ_count["student_name"]))*100
schools_summ_count["% Overall Passing Rate"] = ((schools_summ_count["% Passing Math"])+(schools_summ_count["% Passing Reading"]))/2

# Create array of which columns to display
col_i = ["school","type","student_name","% Passing Math","% Passing Reading","% Overall Passing Rate"]
schools_summ_count = schools_summ_count[col_i]

# Rename columns 
schools_summ_count.rename(columns={"school":"School Name",
                                   "type":"School Type",
                                   "student_name":"Total Students"},inplace=True)                                                    

In [451]:
# Use mean to calculate Total Budget, Average Math Score, Average Reading Score
schools_summ_mean = groupby_table[["budget","math_score","reading_score"]].mean()
schools_summ_mean["Per Student Budget"] = schools_summ_mean["budget"]/schools_summ_count["Total Students"]

# Rename columns
schools_summ_mean.rename(columns={"school":"School Name",
                                  "type":"School Type",
                                  "budget":"Total School Budget", 
                                  "math_score":"Average Math Score",
                                  "reading_score":"Average Reading Score"},inplace=True)

In [452]:
# Merge Count and Mean tables on School Name and School Type
schools_summary = pd.merge(schools_summ_count,schools_summ_mean,how="left",on=["School Name","School Type"])

# Create Array of column names to display
summ_col = ["School Name","School Type","Total Students","Total School Budget","Per Student Budget",
           "Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"]
schools_summary = schools_summary[summ_col]

# Set School Name as index and index name=None
schools_summary.set_index("School Name", drop=False, inplace=True)
schools_summary.index.name = None

# Format column names 
schools_summary["Total Students"] = schools_summary["Total Students"].map("{:,}".format)
schools_summary["Total School Budget"] = schools_summary["Total School Budget"].map("${:,.2f}".format)
#schools_summary['Per Student Budget'] = schools_summary['Per Student Budget'].map('${:,.2f}'.format)
#schools_summary['Average Math Score'] = schools_summary['Average Math Score'].map('{:,.2f}'.format)
#schools_summary['Average Reading Score'] = schools_summary['Average Reading Score'].map('{:,.2f}'.format)
#schools_summary['% Passing Math'] = (schools_summary['% Passing Math']).map('{:,.2f}%'.format)
#schools_summary['% Passing Reading'] = (schools_summary['% Passing Reading']).map('{:,.2f}%'.format)
#schools_summary['% Overall Passing Rate'] = (schools_summary['% Overall Passing Rate']).map('{:,.2f}%'.format)

# Drop School Name for cleaner look
schools_summary_clean = schools_summary.drop('School Name', axis=1)
schools_summary_clean

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,"4,976","$3,124,928.00",628.0,77.048432,81.033963,77.913987,94.553859,86.233923
Cabrera High School,Charter,"1,858","$1,081,356.00",582.0,83.061895,83.975780,100.000000,100.000000,100.000000
Figueroa High School,District,"2,949","$1,884,411.00",639.0,76.711767,81.158020,77.178705,94.540522,85.859613
Ford High School,District,"2,739","$1,763,916.00",644.0,77.102592,80.746258,78.203724,93.866375,86.035049
Griffin High School,Charter,"1,468","$917,500.00",625.0,83.351499,83.816757,100.000000,100.000000,100.000000
Hernandez High School,District,"4,635","$3,022,020.00",652.0,77.289752,80.934412,77.734628,94.606257,86.170442
Holden High School,Charter,427,"$248,087.00",581.0,83.803279,83.814988,100.000000,100.000000,100.000000
Huang High School,District,"2,917","$1,910,635.00",655.0,76.629414,81.182722,77.716832,94.480631,86.098732
Johnson High School,District,"4,761","$3,094,650.00",650.0,77.072464,80.966394,77.966814,94.475950,86.221382
Pena High School,Charter,962,"$585,858.00",609.0,83.839917,84.044699,100.000000,100.000000,100.000000


# Top Performing Schools (By Passing Rate)

In [453]:
# Because there are multiple schools with Overall Passing Rate of 100%, I use a combined average of math and reading scores as the second sort
schools_summary["combined_average"] = (schools_summary["Average Math Score"]+schools_summary["Average Reading Score"])/2
# Sort schools by Overall Passing Rate and Combined Average score
sort_schools = schools_summary.sort_values(["% Overall Passing Rate","combined_average"], ascending=False)
# Find the 5 top performing schools
top_schools = sort_schools.nlargest(5,"% Overall Passing Rate",keep='first')
# Drop columns not needed in final output
top_schools = top_schools.drop(["combined_average","School Name"],axis=1)
top_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Pena High School,Charter,962,"$585,858.00",609.0,83.839917,84.044699,100.0,100.0,100.0
Wright High School,Charter,"1,800","$1,049,400.00",583.0,83.682222,83.955000,100.0,100.0,100.0
Holden High School,Charter,427,"$248,087.00",581.0,83.803279,83.814988,100.0,100.0,100.0
Thomas High School,Charter,"1,635","$1,043,130.00",638.0,83.418349,83.848930,100.0,100.0,100.0
Wilson High School,Charter,"2,283","$1,319,574.00",578.0,83.274201,83.989488,100.0,100.0,100.0


# Bottom Performing Schools (By Passing Rate)

In [454]:
# Find the 5 bottom performing schools by Overall Passing Rate
bottom_schools = sort_schools.nsmallest(5,"% Overall Passing Rate",keep="last")
# Drop columns not needed in final output
bottom_schools = bottom_schools.drop(["combined_average","School Name"],axis=1)
bottom_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Figueroa High School,District,"2,949","$1,884,411.00",639.0,76.711767,81.158020,77.178705,94.540522,85.859613
Ford High School,District,"2,739","$1,763,916.00",644.0,77.102592,80.746258,78.203724,93.866375,86.035049
Huang High School,District,"2,917","$1,910,635.00",655.0,76.629414,81.182722,77.716832,94.480631,86.098732
Hernandez High School,District,"4,635","$3,022,020.00",652.0,77.289752,80.934412,77.734628,94.606257,86.170442
Johnson High School,District,"4,761","$3,094,650.00",650.0,77.072464,80.966394,77.966814,94.475950,86.221382


# Math Scores by Grade

In [455]:
# Create pivot_table for average math_schore by school and grade
math_byGrade = students_df.pivot_table(values='math_score',index='school',columns=['grade'],aggfunc=np.mean)
# Do not show School Name index name
math_byGrade.index.name = None
# Create Array of column names to display
column_order = ['9th', '10th', '11th','12th']
math_byGrade = math_byGrade.reindex_axis(column_order, axis=1)
math_byGrade

grade,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


# Reading Scores by Grade

In [456]:
# Create pivot_table for average reading_score by school and grade
reading_byGrade = students_df.pivot_table(values='reading_score',index='school',columns=['grade'],aggfunc=np.mean)
# Do not show School Name index name
reading_byGrade.index.name = None
# Create Array of column names to display
column_order = ['9th', '10th', '11th','12th']
reading_byGrade = reading_byGrade[column_order]
reading_byGrade

grade,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


# Scores by School Spending

In [457]:
# Bins for School Spending
bins = [0, 585, 615, 645, 675]
group_names = ['<$585', '$585-615', '$615-645', '$645-675']
schools_summary['Spending Ranges (Per Student)'] = pd.cut(schools_summary['Per Student Budget'], 
                                                          bins, labels=group_names)
spend_range_df = pd.merge(merge_table, schools_summary, how='left', left_on='school', right_on='School Name')

# Calculate mean of values by Spending Ranges
spend_range_mean = spend_range_df.groupby('Spending Ranges (Per Student)').mean()

# Create Array of column names to display
col_spend_summ = ['Average Math Score','Average Reading Score',
                  '% Passing Math','% Passing Reading','% Overall Passing Rate']
spend_range_summary = spend_range_mean[col_spend_summ]
spend_range_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.363065,83.964039,100.000000,100.000000,100.000000
$585-615,83.529196,83.838414,100.000000,100.000000,100.000000
$615-645,78.061635,81.434088,81.701002,95.412586,88.556794
$645-675,77.049297,81.005604,77.820190,94.526111,86.173150


# Scores by School Size

In [458]:
# Bins for School Size 
bins = [0, 1000, 3000, 5000]
group_names = ['Small (<1000)', 'Medium (1000-3000)', 'Large (3000-5000)']
merge_table['School Size'] = pd.cut(merge_table['size'], bins, labels=group_names)
school_size_df = pd.merge(merge_table, schools_summary, how='left', left_on='school', right_on='School Name')

# Calculate mean of values by School Size
school_size_mean = school_size_df.groupby('School Size').mean()

# Create Array of column names to display
col_size_summ = ['Average Math Score','Average Reading Score',
                  '% Passing Math','% Passing Reading','% Overall Passing Rate']
school_size_summary = school_size_mean[col_size_summ]
school_size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.828654,83.974082,100.000000,100.000000,100.000000
Medium (1000-3000),80.450902,82.626481,90.108192,97.475528,93.791860
Large (3000-5000),77.070764,80.928365,77.889064,94.562082,86.225573


# Scores by School Type

In [459]:
# Create Array of column names to display
col_type_summ = ['School Type','Average Math Score','Average Reading Score',
                '% Passing Math','% Passing Reading','% Overall Passing Rate']
school_type_summ = schools_summary[col_type_summ]

# Group by School Type and find mean of values
school_type_summ = school_type_summ.groupby('School Type').mean()
school_type_summ

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,100.000000,100.000000,100.00000
District,76.956733,80.966636,77.808454,94.449607,86.12903
